Data is from [this](https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/overview) dataset.

The goal is to predict the daily consumer demand for fresh bakery products on the shelves of over 1 milion stores along 45.000 routes across Mexico.

## A Basic Model as a starting point

Model selection and model fitting is not our goal here

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os

plt.style.use('fivethirtyeight')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Load Data
data_dir = "/content"
data = pd.read_csv(data_dir / "train.csv")
clientes = pd.read_csv(data_dir / "cliente_tabla.csv")
productos = pd.read_csv(data_dir / "producto_tabla.csv")
town_state = pd.read_csv(data_dir / "town_state.csv")

In [ ]:
# Merge datasets
data = data.merge(clientes, on="Cliente_ID", how="left")
data = data.merge(productos, on="Producto_ID", how="left")
data = data.merge(town_state, on="Agencia_ID", how="left")

data.head()

In [ ]:
# Some preprocessing

# Define the categorical columns
categorical_cols = ["Agencia_ID", "Canal_ID", "Ruta_SAK", "Cliente_ID", "Producto_ID"]

# Define the label encoder
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.transform(data[col])
    label_encoders[col] = le

num_unique_vals = {col: data[col].nunique() for col in categorical_cols}
embedding_sizes = {col: min(50, num_unique_vals[col] // 2) for col in categorical_cols}

In [ ]:
# Split into features and target
X = data[categorical_cols].values
y = data["Demanda_uni_equil"].values

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Define the Dataset class
class BimboDataset(Dataset):
    def __init__(self, X, y):
        self.X = [torch.tensor(X[:, i], dtype=torch.long) for i in range(X.shape[1])]
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return [x[idx] for x in self.X], self.y[idx]

# Create Datasets and DataLoaders
train_dataset = BimboDataset(X_train, y_train)
val_dataset = BimboDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

In [ ]:
# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self, embedding_sizes, hidden_size=128):
        super(SimpleModel, self).__init__()
        self.embeddings = nn.ModuleList(
            [
                nn.Embedding(num_unique_vals[col], embedding_sizes[col])
                for col in categorical_cols
            ])
        self.fc1 = nn.Linear(sum(embedding_sizes.values()), hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = [embedding(x_i) for x_i, embedding in zip(x, self.embeddings)]
        x = torch.cat(x, dim=-1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x).squeeze(-1)
        return x


# Define a function to train the modle
# Will do it outside the function
def train_model(loss_fn, num_epochs=5):
    model = SimpleModel(embedding_sizes)
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_targets = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs).squeeze()
                loss = loss_fn(outputs, targets)
                val_loss += loss.item()
                val_preds.extend(outputs.tolist())
                val_targets.extend(targets.tolist())

        val_loss /= len(val_loader)
        r2 = r2_score(val_targets, val_preds)

    return model, np.array(val_preds), np.array(val_targets)

In [ ]:
def get_business_metrics(stocking_decisions, actual_demand):

    frac_understocks = (stocking_decisions < actual_demand).mean()
    total_understocked_amt = (actual_demand - stocking_decisions).clip(0).sum()
    frac_overstocks = (stocking_decisions > actual_demand).mean()
    total_overstocked_amt = (stocking_decisions - actual_demand).clip(0).sum()

    utility = -3 * total_understocked_amt - total_overstocked_amt
    mae = mean_absolute_error(actual_demand, stocking_decisions)
    mse = mean_squared_error(actual_demand, stocking_decisions)
    r2 = r2_score(actual_demand, stocking_decisions)

    # add them in a dictionary
    metrics = {
        'frac_understocks': frac_understocks,
        'total_understocked_amt': total_understocked_amt,
        'frac_overstocks': frac_overstocks,
        'total_overstocked_amt': total_overstocked_amt,
        'utility': utility,
        'mae': mae,
        'mse': mse,
        'r2': r2}

    return metrics

In [ ]:
# Define some train-specifics parameters
loss_fn = nn.MSELoss()
num_epochs = 5
model = SimpleModel(embedding_sizes)
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    # Validation loop
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs).squeeze()
            loss = loss_fn(outputs, targets)
            val_loss += loss.item()
            val_preds.extend(outputs.tolist())
            val_targets.extend(targets.tolist())

    val_loss /= len(val_loader)
    r2 = r2_score(val_targets, val_preds)


# copy the final results
mse_model, mse_val_preds, mse_val_targets = model, np.array(val_preds), np.array(val_targets)
mse_val_stock = np.ceil(mse_val_preds)

In [ ]:
# Get the model results
get_business_metrics(mse_val_stock, mse_val_targets)

In [ ]:
# Try a different decision for re-stocking
alternative_stocking_rule = np.ceil(1.5 * mse_val_preds)
get_business_metrics(alternative_stocking_rule, mse_val_targets)

In [ ]:
# Now train using MAE
loss_fn = nn.L1Loss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    # Validation loop
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs).squeeze()
            loss = loss_fn(outputs, targets)
            val_loss += loss.item()
            val_preds.extend(outputs.tolist())
            val_targets.extend(targets.tolist())

    val_loss /= len(val_loader)
    r2 = r2_score(val_targets, val_preds)


# copy the final results
mae_model, mae_val_preds, mae_val_targets = model, np.array(val_preds), np.array(val_targets)
mae_val_stock = np.ceil(mae_val_preds)

In [ ]:
get_business_metrics(mae_val_stock, mae_val_targets)

Lets assume understock costs 3€ per unit and overstock costs 1€ per unit

In [ ]:
# Define a custom loss function
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, outputs, actual):
        diff = outputs - actual
        loss = torch.where(outputs > actual, diff, -3 * diff)
        return loss.mean()

In [ ]:
# Train the model with the new loss
loss_fn = CustomLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    # Validation loop
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs).squeeze()
            loss = loss_fn(outputs, targets)
            val_loss += loss.item()
            val_preds.extend(outputs.tolist())
            val_targets.extend(targets.tolist())

    val_loss /= len(val_loader)
    r2 = r2_score(val_targets, val_preds)


# copy the final results
custom_model, custom_val_preds, custom_val_targets = model, np.array(val_preds), np.array(val_targets)
custom_val_stock  = np.ceil(custom_val_preds)

In [ ]:
# Then go Streamlit!